In [1]:
from selenium.webdriver import Firefox, FirefoxOptions
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from unidecode import unidecode
import pandas as pd
import numpy as np

# Betway

In [242]:
def get_betway_odds():
    
    opts = FirefoxOptions()
    opts.headless = True
    browser = Firefox(options=opts)
    browser.implicitly_wait(2)

    browser.get('https://betway.com/pt/sports/grp/soccer/brazil/brasileiro-serie-a')
    
    base_xpath = '/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[3]/div[2]/div'

    games_ids = []

    mandantes = []
    visitantes = []

    odds_mandantes = []
    odds_empates = []
    odds_visitantes = []

    for day in range(1,5):
        for game in range(1,10):
            try:
                text_game = browser.find_element(
                    by='xpath',
                    value = base_xpath + f'/div[{day}]/div[2]/div/div[{game}]'
                ).text

                _, game_name, odd_mandante, odd_empate, odd_visitante = text_game.split('\n')
                clubs = [unidecode(club.strip()).lower().replace(' ', '_') for club in game_name.split('-')]
                game_id = '-'.join(sorted(clubs))
                mandante = clubs[0]
                visitante = clubs[1]



                games_ids.append(game_id)
                mandantes.append(mandante)
                visitantes.append(visitante)
                odds_mandantes.append(float(odd_mandante.replace(',', '.')))
                odds_empates.append(float(odd_empate.replace(',', '.')))
                odds_visitantes.append(float(odd_visitante.replace(',', '.')))
            except NoSuchElementException:
                break
    df = pd.DataFrame({
        'game_id': games_ids,
        'mandante': mandantes,
        'visitante': visitantes,
        'odd_mandante': odds_mandantes,
        'odd_empate': odds_empates,
        'odd_visitante': odds_visitantes
    })
    browser.quit()
    
    return df

In [243]:
betway_odds_df = get_betway_odds()

# Sporting Bet

In [256]:
def get_sporting_bet_odds():
    
    clubs_map = {
        'fluminense_fc_rj': 'fluminense',
        'ceara_sc': 'ceara',
        'botafogo_fr_rj': 'botafogo',
        'cuiaba_esporte_clube_mt': 'cuiaba',
        'atletico_mineiro_mg': 'atletico_mineiro',
        'coritiba_fc_pr': 'coritiba',
        'sao_paulo_fc_sp': 'sao_paulo',
        'ca_paranaense_pr': 'athletico_pr',
        'ec_juventude_rs': 'juventude',
        'america_mg': 'america_mineiro',
        'sc_internacional_rs': 'internacional',
        'goias_ec_go': 'goias',
        'ac_goianiense_go': 'atletico_goianiense',
        'santos_fc_sp': 'santos',
        'avai_fc_sc': 'avai',
        'red_bull_bragantino': 'bragantino',
        'cr_flamengo_rj': 'flamengo',
        'sc_corinthians_sp': 'corinthians',
        'fortaleza_ec_ce': 'fortaleza',
        'se_palmeiras_sp': 'palmeiras'
    }
    
    opts = FirefoxOptions()
    opts.headless = True
    browser = Firefox(options=opts)
    browser.implicitly_wait(2)

    browser.get('https://sports.sportingbet.com/en/sports/football-4/betting/brazil-33/brasileiro-serie-a-102838')
    
    base_xpath = '//*[@id="main-view"]/ms-widget-layout/ms-widget-slot/ms-composable-widget/ms-widget-slot[2]/ms-tabbed-grid-widget/ms-grid/div'

    games_ids = []

    mandantes = []
    visitantes = []

    odds_mandantes = []
    odds_empates = []
    odds_visitantes = []

    for day in range(1,5):
        for game in range(1,10):
            try:
                text_game = browser.find_element(
                    by='xpath',
                    value = base_xpath + f'/ms-event-group[{day}]/ms-event[{game}]/div'
                ).text
                
                if len(text_game.split('\n')) == 10:
                    mandante, visitante, _, _, odd_mandante, odd_empate, odd_visitante, _, _, _ = text_game.split('\n')
                elif len(text_game.split('\n')) == 9:
                    mandante, visitante, _, odd_mandante, odd_empate, odd_visitante, _, _, _ = text_game.split('\n')
                elif len(text_game.split('\n')) == 8:
                    mandante, visitante, odd_mandante, odd_empate, odd_visitante, _, _, _ = text_game.split('\n')
                else: 
                    raise NoSuchElementException()

                clubs = [unidecode(club.strip()).lower().replace(' ', '_') for club in [mandante.strip(), visitante.strip()]]
                clubs = [clubs_map[club] if club in clubs_map.keys() else club for club in clubs]
                game_id = '-'.join(sorted(clubs))
                
                mandante = clubs[0]
                visitante = clubs[1]

                games_ids.append(game_id)
                mandantes.append(mandante)
                visitantes.append(visitante)
                odds_mandantes.append(float(odd_mandante))
                odds_empates.append(float(odd_empate))
                odds_visitantes.append(float(odd_visitante))
            
            except NoSuchElementException:
                break
            
    df = pd.DataFrame({
        'game_id': games_ids,
        'mandante': mandantes,
        'visitante': visitantes,
        'odd_mandante': odds_mandantes,
        'odd_empate': odds_empates,
        'odd_visitante': odds_visitantes
    })
    browser.quit()
    
    return df

In [257]:
sporting_bet_df = get_sporting_bet_odds()

In [258]:
sporting_bet_df

,game_id,mandante,visitante,odd_mandante,odd_empate,odd_visitante
0,ceara-fluminense,ceara,fluminense,2.87,3.25,2.50
1,botafogo-cuiaba,botafogo,cuiaba,1.93,3.30,4.10
2,atletico_mineiro-sao_paulo,sao_paulo,atletico_mineiro,2.40,3.20,3.00
3,america_mineiro-internacional,america_mineiro,internacional,2.70,3.10,2.75
4,athletico_pr-goias,athletico_pr,goias,1.55,3.90,6.25
5,coritiba-juventude,juventude,coritiba,2.90,3.20,2.50
6,atletico_goianiense-santos,atletico_goianiense,santos,2.35,3.20,3.10
7,avai-bragantino,avai,bragantino,3.25,3.30,2.20
8,corinthians-flamengo,flamengo,corinthians,1.53,4.00,5.25
9,fortaleza-palmeiras,palmeiras,fortaleza,1.41,4.60,7.75


In [302]:
dfs = [betway_odds_df, sporting_bet_df]
prob_df = get_prob_values(dfs)

In [303]:
prob_df

,game_id,prob_value
0,ceara-fluminense,1.056124
1,atletico_goianiense-santos,1.060613
2,corinthians-flamengo,1.094071
3,avai-bragantino,1.065268
4,botafogo-cuiaba,1.065067
5,fortaleza-palmeiras,1.055643
6,america_mineiro-internacional,1.056587
7,coritiba-juventude,1.057328
8,athletico_pr-goias,1.061572
9,atletico_mineiro-sao_paulo,1.062500


# 365 BET

In [ ]:
def get_365bet_odds():
    
    clubs_map = {
        'fluminense_fc_rj': 'fluminense',
        'ceara_sc': 'ceara',
        'botafogo_fr_rj': 'botafogo',
        'cuiaba_esporte_clube_mt': 'cuiaba',
        'atletico_mineiro_mg': 'atletico_mineiro',
        'coritiba_fc_pr': 'coritiba',
        'sao_paulo_fc_sp': 'sao_paulo',
        'ca_paranaense_pr': 'athletico_pr',
        'ec_juventude_rs': 'juventude',
        'america_mg': 'america_mineiro',
        'sc_internacional_rs': 'internacional',
        'goias_ec_go': 'goias',
        'ac_goianiense_go': 'atletico_goianiense',
        'santos_fc_sp': 'santos',
        'avai_fc_sc': 'avai',
        'red_bull_bragantino': 'bragantino',
        'cr_flamengo_rj': 'flamengo',
        'sc_corinthians_sp': 'corinthians',
        'fortaleza_ec_ce': 'fortaleza',
        'se_palmeiras_sp': 'palmeiras'
    }
    
    opts = FirefoxOptions()
    opts.headless = True
    browser = Firefox(options=opts)
    browser.implicitly_wait(2)

    browser.get('https://sports.sportingbet.com/en/sports/football-4/betting/brazil-33/brasileiro-serie-a-102838')
    
    base_xpath = '//*[@id="main-view"]/ms-widget-layout/ms-widget-slot/ms-composable-widget/ms-widget-slot[2]/ms-tabbed-grid-widget/ms-grid/div'

    games_ids = []

    mandantes = []
    visitantes = []

    odds_mandantes = []
    odds_empates = []
    odds_visitantes = []

    for day in range(1,5):
        for game in range(1,10):
            try:
                text_game = browser.find_element(
                    by='xpath',
                    value = base_xpath + f'/ms-event-group[{day}]/ms-event[{game}]/div'
                ).text
                
                if len(text_game.split('\n')) == 10:
                    mandante, visitante, _, _, odd_mandante, odd_empate, odd_visitante, _, _, _ = text_game.split('\n')
                elif len(text_game.split('\n')) == 9:
                    mandante, visitante, _, odd_mandante, odd_empate, odd_visitante, _, _, _ = text_game.split('\n')
                elif len(text_game.split('\n')) == 8:
                    mandante, visitante, odd_mandante, odd_empate, odd_visitante, _, _, _ = text_game.split('\n')
                else: 
                    raise NoSuchElementException()

                clubs = [unidecode(club.strip()).lower().replace(' ', '_') for club in [mandante.strip(), visitante.strip()]]
                clubs = [clubs_map[club] if club in clubs_map.keys() else club for club in clubs]
                game_id = '-'.join(sorted(clubs))
                
                mandante = clubs[0]
                visitante = clubs[1]

                games_ids.append(game_id)
                mandantes.append(mandante)
                visitantes.append(visitante)
                odds_mandantes.append(float(odd_mandante))
                odds_empates.append(float(odd_empate))
                odds_visitantes.append(float(odd_visitante))
            
            except NoSuchElementException:
                break
            
    df = pd.DataFrame({
        'game_id': games_ids,
        'mandante': mandantes,
        'visitante': visitantes,
        'odd_mandante': odds_mandantes,
        'odd_empate': odds_empates,
        'odd_visitante': odds_visitantes
    })
    browser.quit()
    
    return df

In [44]:
opts = FirefoxOptions()
opts.headless = True
browser = Firefox(options=opts)
browser.implicitly_wait(2)

browser.get('https://www.bet365.com/#/AC/B1/C1/D1002/E71022033/G40/')

In [45]:
base_xpath = '/html/body/div[1]/div/div[3]/div[3]/div/div/div/div[1]/div/div/div[2]/div/div/div[2]/div[2]/div'

browser.find_element(
    by=By.XPATH,
    value = base_xpath
).text

NoSuchElementException: Message: Unable to locate element: /html/body/div[1]/div/div[3]/div[3]/div/div/div/div[1]/div/div/div[2]/div/div/div[2]/div[2]/div
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.jsm:12:1
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:192:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:404:5
element.find/</<@chrome://remote/content/marionette/element.js:291:16


In [ ]:
day = 1
game = 1

browser.find_element(
    by='xpath',
    value = base_xpath + f'/ms-event-group[{day}]/ms-event[{game}]/div'
).text

In [316]:
browser.quit()

# Betfair

In [72]:
opts = FirefoxOptions()
#opts.headless = True
browser = Firefox(options=opts)
browser.implicitly_wait(2)

browser.get('https://www.betfair.com/sport/football')

In [75]:
browser.find_element(
    by=By.XPATH,
    value = '//*[@id="onetrust-accept-btn-handler"]'
).click

<bound method WebElement.click of <selenium.webdriver.remote.webelement.WebElement (session="ea1375e3-67ae-4a00-b03b-a87f7aaf4169", element="b9899a25-dc13-4667-aae0-5de1e0967cae")>>

In [76]:
browser.find_element(
    by=By.XPATH,
    value = '//*[@id="yui_3_5_0_1_1667184721635_52127"]'
).click

<bound method WebElement.click of <selenium.webdriver.remote.webelement.WebElement (session="ea1375e3-67ae-4a00-b03b-a87f7aaf4169", element="f081d1fa-3ad2-41af-b3d9-476af5ae56de")>>

In [79]:
base_xpath = '//*[@id="yui_3_5_0_1_1667184721635_74434"]'

browser.find_element(
    by=By.XPATH,
    value = base_xpath
).text

'Amanhã 20:00\n2.2\n1.6\n2.95\n3.1\n2.5\nSerá Disponibilizado Ao Vivo\nCeará\nFluminense'

In [80]:
base_xpath = '//*[@id="yui_3_5_0_1_1667184721635_74166"]'

browser.find_element(
    by=By.XPATH,
    value = base_xpath
).text

'Amanhã\n1\nX\n2\nMais de\nMenos de\nAmanhã 20:00\n2.2\n1.6\n2.95\n3.1\n2.5\nSerá Disponibilizado Ao Vivo\nCeará\nFluminense\nTerça-feira, 01 Novembro\n1\nX\n2\nMais de\nMenos de\n01 nov 19:00\n2.25\n1.58\n1.85\n3.5\n4.25\nSerá Disponibilizado Ao Vivo\nBotafogo\nCuiabá\n01 nov 21:30\n2.3\n1.56\n2.5\n3.0\n3.0\nSerá Disponibilizado Ao Vivo\nSão Paulo\nAtlético-MG\nQuarta-feira, 02 Novembro\n1\nX\n2\nMais de\nMenos de\n02 nov 16:00\n2.25\n1.58\n2.6\n3.0\n2.8\nSerá Disponibilizado Ao Vivo\nAmérica-MG\nInternacional\n02 nov 16:00\n1.95\n1.78\n1.5\n4.0\n6.0\nSerá Disponibilizado Ao Vivo\nAthletico-PR\nGoiás\n02 nov 19:00\n2.15\n1.65\n3.25\n3.25\n2.25\nSerá Disponibilizado Ao Vivo\nAvaí\nBragantino\n02 nov 19:00\n2.2\n1.6\n3.0\n3.25\n2.4\nSerá Disponibilizado Ao Vivo\nJuventude\nCoritiba\n02 nov 19:00\n2.2\n1.6\n2.3\n3.0\n3.25\nSerá Disponibilizado Ao Vivo\nAtlético-GO\nSantos\n02 nov 21:30\n1.78\n1.95\n1.36\n4.75\n8.0\nSerá Disponibilizado Ao Vivo\nPalmeiras\nFortaleza\n02 nov 21:30\n2.0\n1.

In [81]:
browser.quit()

# Main

In [29]:
from surebet.utils import (
    get_prob_values,
    get_game_ids
)
from src.runners.main_br_serie_a import get_br_serie_a_odds

In [57]:
def get_prob_values(dfs: pd.DataFrame, empate: bool = True) -> pd.DataFrame:
    """Retorna os valores de probalidade para o surebet.

    Args:
        dfs (pd.DataFrame): Scrapers
        empate (bool): Caso ha possibildiade de empate

    Returns:
        pd.DataFrame: Dataframe com game_id, prob_value
    """
    #logger.info('Getting Prob Values for Surebet...')

    game_ids = get_game_ids(dfs)
    all_odds_df = pd.concat(odds_df)

    prob_df = pd.DataFrame()

    for game_id in game_ids:

        df_game = all_odds_df.query('game_id == @game_id')

        max_odd_mandante = (
            df_game
            .nlargest(1, 'odd_mandante')
            .iloc[0]['odd_mandante']
        )
        max_odd_visitante = (
            df_game
            .nlargest(1, 'odd_visitante')
            .iloc[0]['odd_visitante']
        )
        if empate:
            max_odd_empate = (
                df_game
                .nlargest(1, 'odd_empate')
                .iloc[0]['odd_empate']
            )

        max_odd_mandante_casa = (
            df_game
            .nlargest(1, 'odd_mandante')
            .iloc[0]['casa_de_aposta']
        )
        max_odd_visitante_casa = (
            df_game
            .nlargest(1, 'odd_visitante')
            .iloc[0]['casa_de_aposta']
        )
        if empate:
            max_odd_empate_casa = (
                df_game
                .nlargest(1, 'odd_empate')
                .iloc[0]['casa_de_aposta']
            )

        prob_mandante = 1/max_odd_mandante
        prob_visitante = 1/max_odd_visitante
        total_prob_value = prob_mandante + prob_visitante
        if empate:
            prob_empate = 1/max_odd_empate
            total_prob_value = total_prob_value + prob_empate

        game_prob_df = pd.DataFrame({
            'game_id': [game_id],
            'total_prob_value': [total_prob_value],
            'mandante': [df_game['mandante'].values[0]],
            'max_odd_mandante': [max_odd_mandante],
            'prob_mandante': [prob_mandante],
            'max_odd_mandante_casa': [max_odd_mandante_casa],
            'max_odd_visitante': [max_odd_visitante],
            'prob_visitante': [prob_visitante],
            'max_odd_visitante_casa': [max_odd_visitante_casa]
        })
        if empate:
            game_prob_df['max_odd_empate'] = max_odd_empate
            game_prob_df['prob_empate'] = prob_empate
            game_prob_df['max_odd_empate_casa'] = max_odd_empate_casa

        prob_df = pd.concat([prob_df, game_prob_df])

    prob_df = (
        prob_df
        .sort_values(by=['total_prob_value'])
    )

    return prob_df

In [4]:
%reload_kedro

UsageError: Line magic function `%reload_kedro` not found.


In [1]:
catalog.list()

NameError: name 'catalog' is not defined

In [3]:
odds_df = get_br_serie_a_odds()

In [39]:
all_odds_df.head()

,game_id,mandante,visitante,odd_mandante,odd_empate,odd_visitante,casa_de_aposta
0,ceara-fluminense,ceara,fluminense,2.87,3.1,2.50,Betway
1,botafogo-cuiaba,botafogo,cuiaba,1.87,3.3,4.20,Betway
2,atletico_mineiro-sao_paulo,sao_paulo,atletico_mineiro,2.50,3.0,2.90,Betway
3,america_mineiro-internacional,america_mineiro,internacional,2.60,3.0,2.75,Betway
4,athletico_pr-goias,athletico_pr,goias,1.55,3.8,5.75,Betway


In [58]:
df = get_prob_values(odds_df)

In [59]:
df

,game_id,total_prob_value,mandante,max_odd_mandante,prob_mandante,max_odd_mandante_casa,max_odd_visitante,prob_visitante,max_odd_visitante_casa,max_odd_empate,prob_empate,max_odd_empate_casa
0,atletico_mineiro-sao_paulo,1.045833,sao_paulo,2.50,0.400000,Betway,3.00,0.333333,Sportingbet,3.2,0.312500,Sportingbet
0,atletico_goianiense-santos,1.050532,atletico_goianiense,2.35,0.425532,Sportingbet,3.20,0.312500,Betway,3.2,0.312500,Sportingbet
0,fortaleza-palmeiras,1.055643,palmeiras,1.41,0.709220,Sportingbet,7.75,0.129032,Sportingbet,4.6,0.217391,Sportingbet
0,america_mineiro-internacional,1.056587,america_mineiro,2.70,0.370370,Sportingbet,2.75,0.363636,Betway,3.1,0.322581,Sportingbet
0,ceara-fluminense,1.057328,ceara,2.90,0.344828,Sportingbet,2.50,0.400000,Betway,3.2,0.312500,Sportingbet
0,coritiba-juventude,1.057328,juventude,2.90,0.344828,Sportingbet,2.50,0.400000,Sportingbet,3.2,0.312500,Betway
0,botafogo-cuiaba,1.059260,botafogo,1.93,0.518135,Sportingbet,4.20,0.238095,Betway,3.3,0.303030,Betway
0,athletico_pr-goias,1.061572,athletico_pr,1.55,0.645161,Betway,6.25,0.160000,Sportingbet,3.9,0.256410,Sportingbet
0,avai-bragantino,1.065268,avai,3.25,0.307692,Sportingbet,2.20,0.454545,Sportingbet,3.3,0.303030,Sportingbet
0,corinthians-flamengo,1.070261,flamengo,1.53,0.653595,Sportingbet,6.00,0.166667,Betway,4.0,0.250000,Betway


In [7]:
odds_df[0].head()

,game_id,mandante,visitante,odd_mandante,odd_empate,odd_visitante,casa_de_aposta
0,ceara-fluminense,ceara,fluminense,2.87,3.1,2.50,Betway
1,botafogo-cuiaba,botafogo,cuiaba,1.87,3.3,4.20,Betway
2,atletico_mineiro-sao_paulo,sao_paulo,atletico_mineiro,2.50,3.0,2.90,Betway
3,america_mineiro-internacional,america_mineiro,internacional,2.60,3.0,2.75,Betway
4,athletico_pr-goias,athletico_pr,goias,1.55,3.8,5.75,Betway


In [15]:
dfs = odds_df.copy()
game_ids = get_game_ids(dfs)
all_odds_df = pd.concat(odds_df)
prob_values = []
for game_id in game_ids:

    df_game = all_odds_df.query('game_id == @game_id')

In [16]:
df_game

,game_id,mandante,visitante,odd_mandante,odd_empate,odd_visitante,casa_de_aposta
7,coritiba-juventude,juventude,coritiba,2.87,3.2,2.37,Betway
5,coritiba-juventude,juventude,coritiba,2.90,3.2,2.50,Sportingbet


In [17]:
df_game.nlargest(1, 'odd_mandante')

,game_id,mandante,visitante,odd_mandante,odd_empate,odd_visitante,casa_de_aposta
5,coritiba-juventude,juventude,coritiba,2.9,3.2,2.5,Sportingbet


In [28]:
df_game.nlargest(1, 'odd_mandante').iloc[0]

game_id           coritiba-juventude
mandante                   juventude
visitante                   coritiba
odd_mandante                     2.9
odd_empate                       3.2
odd_visitante                    2.5
casa_de_aposta           Sportingbet
Name: 5, dtype: object

In [12]:
pd.concat(odds_df)

,game_id,mandante,visitante,odd_mandante,odd_empate,odd_visitante,casa_de_aposta
0,ceara-fluminense,ceara,fluminense,2.87,3.10,2.50,Betway
1,botafogo-cuiaba,botafogo,cuiaba,1.87,3.30,4.20,Betway
2,atletico_mineiro-sao_paulo,sao_paulo,atletico_mineiro,2.50,3.00,2.90,Betway
3,america_mineiro-internacional,america_mineiro,internacional,2.60,3.00,2.75,Betway
4,athletico_pr-goias,athletico_pr,goias,1.55,3.80,5.75,Betway
5,atletico_goianiense-santos,atletico_goianiense,santos,2.30,3.00,3.20,Betway
6,avai-bragantino,avai,bragantino,3.20,3.20,2.15,Betway
7,coritiba-juventude,juventude,coritiba,2.87,3.20,2.37,Betway
8,corinthians-flamengo,flamengo,corinthians,1.50,4.00,6.00,Betway
9,fortaleza-palmeiras,palmeiras,fortaleza,1.40,4.33,7.00,Betway


In [14]:
pd.concat(odds_df).nlargest(1,'odd_mandante')

,game_id,mandante,visitante,odd_mandante,odd_empate,odd_visitante,casa_de_aposta
7,avai-bragantino,avai,bragantino,3.25,3.3,2.2,Sportingbet
